In [354]:
import geopandas as gpd
import pandas as pd
import numpy as np

gdf = gpd.read_file("/data/MASTER_dataset CLEANED.geojson")

# Safety: ensure GEOID column name is consistent
if "GEOID" in gdf.columns and "geoid" not in gdf.columns:
    gdf = gdf.rename(columns={"GEOID": "geoid"})

In [355]:
gdf = gdf.copy()
gdf["geometry"] = gdf.geometry.make_valid()

# Ensure CRS is set
if gdf.crs is None:
    raise ValueError("Master gdf has no CRS. Set it before doing spatial joins.")

In [356]:
# Create inside-points (NOT centroids)
pts = gdf[["geoid", "tract_land_type", "geometry"]].copy()
pts["geometry"] = pts.geometry.representative_point()
pts = pts.set_crs(gdf.crs)

In [357]:
# CITY JOIN
places = gpd.read_file("/data/flcities/tl_2025_12_place.shp")[["NAME", "NAMELSAD", "geometry"]]
places = places.to_crs(gdf.crs)

In [358]:
# City join — WITHIN (land + mostly_water)
pts_city = pts[pts["tract_land_type"].isin(["land", "mostly_water"])].copy()

city_within = gpd.sjoin(
    pts_city[["geoid", "geometry"]],
    places,
    how="left",
    predicate="within"
).drop(columns=["index_right"])

city_within = city_within.rename(columns={"NAME": "city_name", "NAMELSAD": "city_label"})[
    ["geoid", "city_name", "city_label"]
]

city_out = pd.DataFrame({"geoid": pts["geoid"], "tract_land_type": pts["tract_land_type"]})
city_out = city_out.merge(city_within, on="geoid", how="left")

city_out["city_method"] = np.where(city_out["city_name"].notna(), "within", pd.NA)
city_out["dist_to_city_m"] = np.nan

In [359]:
# City join — NEAREST fallback (LAND only, thresholded)
missing_land_city = (city_out["tract_land_type"] == "land") & (city_out["city_name"].isna())

if missing_land_city.any():
    projected_crs = "EPSG:3857"
    MAX_CITY_DIST_M = 3000

    pts_missing = pts.loc[pts["geoid"].isin(city_out.loc[missing_land_city, "geoid"]), ["geoid", "geometry"]].copy()
    pts_missing_p = pts_missing.to_crs(projected_crs)
    places_p = places.to_crs(projected_crs)

    city_nn = gpd.sjoin_nearest(
        pts_missing_p,
        places_p,
        how="left",
        distance_col="dist_to_city_m"
    ).drop(columns=["index_right"])

    city_nn = city_nn.rename(columns={"NAME": "city_name_nn", "NAMELSAD": "city_label_nn"})[
        ["geoid", "city_name_nn", "city_label_nn", "dist_to_city_m"]
    ]

    city_nn["ok"] = city_nn["dist_to_city_m"] <= MAX_CITY_DIST_M

    city_out = city_out.merge(city_nn, on="geoid", how="left")

    fill = missing_land_city & city_out["ok"].infer_objects(copy=False)
    city_out.loc[fill, "city_name"] = city_out.loc[fill, "city_name_nn"]
    city_out.loc[fill, "city_label"] = city_out.loc[fill, "city_label_nn"]
    city_out.loc[fill, "city_method"] = "nearest"

    # Keep dist_to_city_m only where nearest used
    city_out.loc[~(city_out["city_method"] == "nearest"), "dist_to_city_m"] = np.nan

    city_out = city_out.drop(columns=["city_name_nn", "city_label_nn", "ok"], errors="ignore")

In [360]:
# City labels — UNINCORPORATED / MOSTLY_WATER / WATER
# LAND still missing -> unincorporated
land_still_missing = (city_out["tract_land_type"] == "land") & (city_out["city_name"].isna())
city_out.loc[land_still_missing, "city_name"] = "Unincorporated Area"
city_out.loc[land_still_missing, "city_method"] = "unincorporated"

# MOSTLY_WATER still missing -> coastal/water-adjacent label
mw_missing = (city_out["tract_land_type"] == "mostly_water") & (city_out["city_name"].isna())
city_out.loc[mw_missing, "city_name"] = "Coastal / Water-adjacent"
city_out.loc[mw_missing, "city_method"] = "mostly_water"

# WATER tracts -> no city
water = (city_out["tract_land_type"] == "water")
city_out.loc[water, "city_name"] = "No city (Water tract)"
city_out.loc[water, "city_method"] = "water"

In [361]:
# ZIP JOIN
zcta = gpd.read_file("/data/zipcodes/tl_2025_us_zcta520.shp")[["ZCTA5CE20", "geometry"]]
zcta = zcta.to_crs(gdf.crs)

In [362]:
# ZIP join — WITHIN (land + mostly_water)
pts_zip = pts[pts["tract_land_type"].isin(["land", "mostly_water"])].copy()

zcta_within = gpd.sjoin(
    pts_zip[["geoid", "geometry"]],
    zcta,
    how="left",
    predicate="within"
).drop(columns=["index_right"])

zcta_within = zcta_within.rename(columns={"ZCTA5CE20": "zcta"})[["geoid", "zcta"]]

zcta_out = pd.DataFrame({"geoid": pts["geoid"], "tract_land_type": pts["tract_land_type"]})
zcta_out = zcta_out.merge(zcta_within, on="geoid", how="left")

zcta_out["zcta_method"] = np.where(zcta_out["zcta"].notna(), "within", pd.NA)
zcta_out["dist_to_zcta_m"] = np.nan

In [363]:
# ZIP join — NEAREST fallback (LAND only, thresholded)
missing_land_zip = (zcta_out["tract_land_type"] == "land") & (zcta_out["zcta"].isna())

if missing_land_zip.any():
    projected_crs = "EPSG:3857"
    MAX_ZCTA_DIST_M = 2000

    pts_missing = pts.loc[pts["geoid"].isin(zcta_out.loc[missing_land_zip, "geoid"]), ["geoid", "geometry"]].copy()
    pts_missing_p = pts_missing.to_crs(projected_crs)
    zcta_p = zcta.to_crs(projected_crs)

    zcta_nn = gpd.sjoin_nearest(
        pts_missing_p,
        zcta_p,
        how="left",
        distance_col="dist_to_zcta_m"
    ).drop(columns=["index_right"])

    zcta_nn = zcta_nn.rename(columns={"ZCTA5CE20": "zcta_nn"})[["geoid", "zcta_nn", "dist_to_zcta_m"]]
    zcta_nn["ok"] = zcta_nn["dist_to_zcta_m"] <= MAX_ZCTA_DIST_M

    zcta_out = zcta_out.merge(zcta_nn, on="geoid", how="left")

    fill = missing_land_zip & zcta_out["ok"].infer_objects(copy=False)
    zcta_out.loc[fill, "zcta"] = zcta_out.loc[fill, "zcta_nn"]
    zcta_out.loc[fill, "zcta_method"] = "nearest"

    zcta_out.loc[~(zcta_out["zcta_method"] == "nearest"), "dist_to_zcta_m"] = np.nan
    zcta_out = zcta_out.drop(columns=["zcta_nn", "ok"], errors="ignore")

In [364]:
# ZIP join — OVERLAP fallback (LAND only, for anything still missing)
still_missing_land_zip = (zcta_out["tract_land_type"] == "land") & (zcta_out["zcta"].isna())

if still_missing_land_zip.any():
    projected_crs = "EPSG:3857"
    missing_geoids = zcta_out.loc[still_missing_land_zip, "geoid"]

    tracts_missing = gdf.loc[gdf["geoid"].isin(missing_geoids), ["geoid", "geometry"]].copy()

    tracts_p = tracts_missing.to_crs(projected_crs)
    zcta_p = zcta.to_crs(projected_crs)

    overlaps = gpd.overlay(tracts_p, zcta_p, how="intersection", keep_geom_type=False)
    overlaps = overlaps[overlaps.geometry.area > 0].copy()
    overlaps["overlap_area_m2"] = overlaps.geometry.area

    best = (
        overlaps.sort_values("overlap_area_m2", ascending=False)
                .drop_duplicates(subset=["geoid"])
                [["geoid", "ZCTA5CE20"]]
                .rename(columns={"ZCTA5CE20": "zcta_overlap"})
    )

    zcta_out = zcta_out.merge(best, on="geoid", how="left")

    fill = still_missing_land_zip & zcta_out["zcta_overlap"].notna()
    zcta_out.loc[fill, "zcta"] = zcta_out.loc[fill, "zcta_overlap"]
    zcta_out.loc[fill, "zcta_method"] = "overlap"

    zcta_out = zcta_out.drop(columns=["zcta_overlap"], errors="ignore")


In [365]:
# ZIP labels — MOSTLY_WATER / WATER
mw_zip_missing = (zcta_out["tract_land_type"] == "mostly_water") & (zcta_out["zcta"].isna())
zcta_out.loc[mw_zip_missing, "zcta"] = "No ZCTA (Mostly water)"
zcta_out.loc[mw_zip_missing, "zcta_method"] = "mostly_water"

water_zip = (zcta_out["tract_land_type"] == "water")
zcta_out.loc[water_zip, "zcta"] = "No ZCTA (Water tract)"
zcta_out.loc[water_zip, "zcta_method"] = "water"

# Final catch-all for land tracts that still have no ZCTA
still_missing_land = (
    (zcta_out["tract_land_type"] == "land") &
    (zcta_out["zcta"].isna())
)

zcta_out.loc[still_missing_land, "zcta_method"] = "unassigned_land"
zcta_out.loc[still_missing_land, "zcta"] = "No ZCTA (Land tract)"

In [366]:
# Merge city + zip back into the master GeoDataFrame
gdf_final = (
    gdf
    .drop(columns=[
        "city_name", "city_label", "city_method", "dist_to_city_m",
        "zcta", "zcta_method", "dist_to_zcta_m"
    ], errors="ignore")
    .merge(
        city_out[["geoid", "city_name", "city_label", "city_method", "dist_to_city_m"]],
        on="geoid",
        how="left"
    )
    .merge(
        zcta_out[["geoid", "zcta", "zcta_method", "dist_to_zcta_m"]],
        on="geoid",
        how="left"
    )
)

In [367]:
gdf_final["zcta"] = gdf_final["zcta"].astype(str).str.zfill(5)

In [368]:
# Validate
city_validation = (
    city_out["city_method"]
    .value_counts(dropna=False)
    .rename_axis("method")
    .reset_index(name="tract_count")
)
city_validation["pct_of_tracts"] = (city_validation["tract_count"] / city_validation["tract_count"].sum() * 100).round(2)

zip_validation = (
    zcta_out["zcta_method"]
    .value_counts(dropna=False)
    .rename_axis("method")
    .reset_index(name="tract_count")
)
zip_validation["pct_of_tracts"] = (zip_validation["tract_count"] / zip_validation["tract_count"].sum() * 100).round(2)

city_validation, zip_validation


(           method  tract_count  pct_of_tracts
 0          within         1276          83.62
 1         nearest          193          12.65
 2  unincorporated           36           2.36
 3    mostly_water           14           0.92
 4           water            7           0.46,
             method  tract_count  pct_of_tracts
 0           within         1509          98.89
 1            water            7           0.46
 2          nearest            4           0.26
 3          overlap            3           0.20
 4  unassigned_land            2           0.13
 5     mostly_water            1           0.07)

In [369]:
bad_geoids = zcta_out.loc[still_missing_land, "geoid"].tolist()
bad_geoids

['12086980100', '12099980400']

In [370]:
projected_crs = "EPSG:3857"

tracts_bad = gdf.loc[gdf["geoid"].isin(bad_geoids), ["geoid", "geometry"]].copy()
tracts_bad = tracts_bad.to_crs(projected_crs)
zcta_p = zcta.to_crs(projected_crs)

# make valid to reduce weird overlay edge cases
tracts_bad["geometry"] = tracts_bad.geometry.make_valid()
zcta_p["geometry"] = zcta_p.geometry.make_valid()

over = gpd.overlay(tracts_bad, zcta_p, how="intersection", keep_geom_type=False)
over = over[over.geometry.area > 0].copy()
over["area_m2"] = over.geometry.area

over.groupby("geoid")["area_m2"].sum().sort_values(ascending=False)

Series([], Name: area_m2, dtype: float64)

In [371]:
# Validate missingness
gdf_final[["city_name", "zcta"]].isna().mean()

city_name    0.0
zcta         0.0
dtype: float64

In [372]:
gdf_final["city_name"].value_counts().head(10)

city_name
Miami                  133
Hialeah                 57
Fort Lauderdale         49
Boca Raton              43
Hollywood               37
Unincorporated Area     36
West Palm Beach         35
Miami Gardens           35
Delray Beach            32
Pembroke Pines          31
Name: count, dtype: int64

In [373]:
gdf_final["zcta"].value_counts().head(10)

zcta
33012    20
33015    18
33186    18
33311    17
33125    16
33157    16
33411    15
33176    15
33130    15
33160    15
Name: count, dtype: int64

In [374]:
gdf_final.sort_values("nri_top3_hazards_eal_total_usd", ascending=False)[
    ["geoid", "city_name", "zcta", "nri_top3_hazards_eal_total_usd"]
].head(20)

,geoid,city_name,zcta,nri_top3_hazards_eal_total_usd
538,12086011500,Unincorporated Area,33034,4.166902e+07
1407,12099007917,Unincorporated Area,33430,2.015150e+07
24,12011010503,Parkland,33076,1.162628e+07
989,12086019600,Country Walk,33196,1.094597e+07
220,12086009010,Doral,33172,9.756605e+06
1523,12087972700,Unincorporated Area,33037,8.896739e+06
1040,12011042502,Fort Lauderdale,33301,5.607886e+06
845,12086010706,Homestead Base,33032,5.514752e+06
259,12086008904,Westchester,33174,5.496209e+06
1401,12099007774,Unincorporated Area,33446,5.465151e+06


In [375]:
gdf_final.to_file(
    "/data/MASTER_dataset CLEANED v2.geojson",
    driver="GeoJSON"
)

In [376]:
gdf_final.drop(columns="geometry").to_csv("/data/MASTER_dataset CLEANED v2.csv", index=False)